# Awesome Maps with Bokeh


---

## Tim Renner
## PyData Meetup 4/25

# About Me
---

## Data Scientist (Geo) @ HomeAway
## Github: [github.com/timothyrenner](http://www.github.com/timothyrenner)

<center>![](https://www.outsideonline.com/sites/default/files/styles/three-quarter-page-scaled-1x/public/bigfoot-patterson-inline.jpg?itok=w4cVFPEB)</center>

## Import data into GeoPandas.

In [22]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

sightings_csv = pd.read_csv('data/california_sightings.csv')

# Use the set_geometry method with shapely Point objects to 
# define the geometry.
sightings = \
    gpd.GeoDataFrame(sightings_csv, geometry = None)\
       .set_geometry(
           # Build a list of Point objects.
           [Point(r['longitude'], r['latitude']) 
            for _,r in sightings_csv.iterrows()],
           # Assign EPSG:4326 Lat/Lon coordinates.
           crs = {"init":"epsg:4326"})

## Define and apply projections onto the dataset.

In [23]:
sightings.to_crs({"init": "epsg:3857"}, inplace=True)

## Make clusters.

In [25]:
from sklearn.neighbors import DistanceMetric
from sklearn.cluster import DBSCAN
import numpy as np

haversine = DistanceMetric.get_metric('haversine')

# Build the distance matrix with the Haversine distance.
sighting_distance_matrix = \
    haversine.pairwise(
        sightings[['latitude','longitude']] * np.pi / 180.0)

# Apply the density based clustering.
dbscan = DBSCAN(eps=0.0075, metric="precomputed")

sightings['cluster'] = \
    dbscan.fit_predict(sighting_distance_matrix)

# Make polygons out of clusters.

In [26]:
cluster_polygons = \
    [
        # unary_union makes a multipoint, 
        # convex_hull draws the polygon.
        group.geometry.unary_union.convex_hull
        for cluster, group 
        in sightings[['cluster','geometry']].groupby('cluster')
        # Cluster -1 is for "noise outliers".
        if cluster != -1
    ]

# Extract the coordinates for Bokeh.

In [27]:
from toolz import get, compose, curry

point_x = [p.coords[0][0] for p in sightings.geometry]
point_y = [p.coords[0][1] for p in sightings.geometry]

# Helpers for the patches.
listmap = compose(list,map)
get_x = curry(get)(0)
get_y = curry(get)(1)

patch_coords = [cp.exterior.coords for cp in cluster_polygons]

# A list of lists of x and y coordinates.
patch_xs = [listmap(get_x, pc) for pc in patch_coords]
patch_ys = [listmap(get_y, pc) for pc in patch_coords]

# Draw the blank map.

In [35]:
import bokeh.plotting as bp
from bokeh.tile_providers import CARTODBPOSITRON_RETINA

x_range = [min(x)+2500000, max(x)-2500000]
y_range = [min(y)+75000, max(y)-75000]

bigfoot_map = \
    bp.figure(
        tools = 'pan,wheel_zoom,reset',
        plot_width = 800, 
        plot_height = 500,
        x_range = x_range, 
        y_range = y_range)

# Turn off the grids and whatnot.
bigfoot_map.axis.visible= False
bigfoot_map.xgrid.grid_line_color = None
bigfoot_map.ygrid.grid_line_color = None

# Add the tiles.
bigfoot_map.add_tile(CARTODBPOSITRON_RETINA)

TileRenderer(id='5d74e222-171a-4560-bc73-7746e5f3a1fa', ...)

# Find Bigfoot.

In [36]:
from bokeh.io import show, output_notebook

output_notebook()

# Add the circles.
bigfoot_map.circle(point_x, point_y, alpha=0.3, size=2, color='black')

# Add the patches.
bigfoot_map.patches(xs = patch_xs, ys = patch_ys, fill_alpha=0.1, color="lightblue")

show(bigfoot_map)

Loading BokehJS ...

# References

----

## Bigfoot Field Research Organization (BFRO): [bfro.net](http://www.bfro.net/)
## Bokeh: [bokeh.pydata.org](http://bokeh.pydata.org/en/latest/)
## Shapely: [github.com/Toblerity/Shapely](https://github.com/Toblerity/Shapely)
## Toolz: [toolz.readthedocs.io](http://toolz.readthedocs.io/en/latest/index.html)
## GeoPandas: [geopandas.org](http://geopandas.org/index.html)
## RISE: [github.com/damianavila/RISE](https://github.com/damianavila/RISE)

# Code
---

## GitHub: [github.com/timothyrenner/bokeh_map_pydata]()